# Configuration Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## Infos

In [ ]:
import sys
sys.version

In [ ]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

In [ ]:
from azureml.core import Workspace, Environment

## Version Azure ML installée

In [ ]:
import azureml.core
print("Version Azure ML =",azureml.core.VERSION)

In [ ]:
pip list | grep azureml

> Versions Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/azure-machine-learning-release-notes

## Workspace Azure ML

In [ ]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "dbd82622-13e7-42bc-9061-a17f74627285") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "rg_amldemo") #Resource group
workspace_name = os.environ.get("WORKSPACE_NAME", "amldemo") #Nom workspace

In [ ]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("Le workspace Azure ML service a été trouvé : OK")
except:
   print("Le workspace Azure ML service n'a pas été trouvé")

In [ ]:
# Lecture du fichier de config
ws = Workspace.from_config()
print('Workspace : ' + ws.name, 
      'Région : ' + ws.location, 
      'Groupe de ressources : ' + ws.resource_group, sep='\n')

In [ ]:
# Infos
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

In [ ]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("- Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("- Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

# Connect Azure Storage

### Prerequisite

Creer un Azure Datalake Gen 2 dans un groupe de resource

How to:https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create?toc=%2Fazure%2Fstorage%2Fblobs%2Ftoc.json&tabs=azure-portal#create-a-storage-account

Create an Azure file share

Documentation
    https://docs.microsoft.com/en-us/azure/storage/files/storage-how-to-create-file-share?tabs=azure-portal


## Connect a Fileshare

In [ ]:
# example for fileshare registration
#

file_datastore_name='azfilesharesdk' # Name of the datastore to workspace
file_share_name=os.getenv("FILE_SHARE_CONTAINER", "fileshare01") # Name of Azure file share container
account_name=os.getenv("FILE_SHARE_ACCOUNTNAME", "mdlegoadls") # Storage account name
account_key=os.getenv("FILE_SHARE_ACCOUNT_KEY", "EKgaDNKjsZtNN4mL1N08UT8PErOTyHbg57wRDUmlRC1Ot5bwm0on9X7BXquorVEX61zjMRsNPr4/5uWfy9MfpA==") # Storage account access key

file_datastore = Datastore.register_azure_file_share(workspace=ws,
                                                     datastore_name=file_datastore_name, 
                                                     file_share_name=file_share_name, 
                                                     account_name=account_name,
                                                     account_key=account_key)

In [ ]:
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)

In [ ]:
from azureml.core import Dataset
   dataset = Dataset.Tabular.from_delimited_files(path = [(azfilesharesdk, 'lego/parts.csv')])

   # preview the first 3 rows of the dataset
   dataset.take(3).to_pandas_dataframe()

# (Optionnel)

## Connect Azure Datalake Store Gen 2

### Create a service principal for ADLS Gen 2

How to: Use the portal to create an Azure AD application and service principal that can access resources
Documentation:
    https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal

Create an Azure service principal with the Azure CLI
Documentation: 
    https://docs.microsoft.com/en-us/cli/azure/create-an-azure-service-principal-azure-cli?view=azure-cli-latest 
    


In [ ]:
# example for Azure Datalake Gen 2

adlsgen2_datastore_name = 'ADLS store name'

subscription_id=os.getenv("ADL_SUBSCRIPTION", "<my_subscription_id>") # subscription id of ADLS account
resource_group=os.getenv("ADL_RESOURCE_GROUP", "<my_resource_group>") # resource group of ADLS account

account_name=os.getenv("ADLSGEN2_ACCOUNTNAME", "<my_account_name>") # ADLS Gen2 account name
tenant_id=os.getenv("ADLSGEN2_TENANT", "<my_tenant_id>") # tenant id of service principal
client_id=os.getenv("ADLSGEN2_CLIENTID", "<my_client_id>") # client id of service principal
client_secret=os.getenv("ADLSGEN2_CLIENT_SECRET", "<my_client_secret>") # the secret of service principal

adlsgen2_datastore = Datastore.register_azure_data_lake_gen2(workspace=ws,
                                                             datastore_name=adlsgen2_datastore_name,
                                                             account_name=account_name, # ADLS Gen2 account name
                                                             filesystem='test', # ADLS Gen2 filesystem
                                                             tenant_id=tenant_id, # tenant id of service principal
                                                             client_id=client_id, # client id of service principal
                                                             client_secret=client_secret) # the secret of service principal

# Documentation

- Documentation :<br> https://docs.microsoft.com/en-us/azure/machine-learning/

- Nouveautés versions Azure ML service: <br>
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes?view=azure-ml-py

- What is Azure ML service SDK?<br>
https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py

- Azure ML service CLI<br>
https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-ml/ml?view=azure-cli-latest

- Azure ML Visual Interface<br>
Algorithm & module reference overview<br>
https://docs.microsoft.com/en-us/azure/machine-learning/algorithm-module-reference/module-reference


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">